In [1]:
pip install tk

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np

In [3]:
import kmeans8 as kmeans

data = kmeans.loadCSV(
    "/scratch/areddy27/dm-hw3-24/kmeans_data/data.csv"
)
label = kmeans.loadCSV(
    "/scratch/areddy27/dm-hw3-24/kmeans_data/label.csv"
)


In [4]:
k = len(set(label))
print(k) 

10


In [5]:
k = len(set(label))  
metrics = [ "jaccard", "euclidean","cosine"]
results = {}

for metric in metrics:
    clustering_result = kmeans.kmeans(data, k, metric)
    results[metric] = clustering_result
    print(f"SSE for {metric}-K-means: {results[metric]['withinss']}")

SSE for jaccard-K-means: 9999.0
SSE for euclidean-K-means: 25400856764.478363
SSE for cosine-K-means: 2456.424805130752


In [6]:
def label_clusters(clusters, labels, df):
    instance_to_label = {tuple(instance): labels[idx] for idx, instance in enumerate(df)}
    cluster_labels = []
    for cluster in clusters:
        if not cluster:
            cluster_labels.append(None)
            continue
        label_counts = {}
        for instance in cluster:
            instance_label = instance_to_label[tuple(instance)]
            label_counts[instance_label] = label_counts.get(instance_label, 0) + 1
        most_frequent_label = max(label_counts, key=label_counts.get)
        cluster_labels.append(most_frequent_label)
    return cluster_labels

def calculate_accuracy(clusters, cluster_labels, actual_labels, df):
    instance_to_actual_label = {tuple(instance): actual_labels[idx] for idx, instance in enumerate(df)}
    correct_assignments = 0
    total_assignments = 0
    for cluster_index, cluster in enumerate(clusters):
        cluster_label = cluster_labels[cluster_index]
        if cluster_label is None:
            continue
        for instance in cluster:
            actual_label = instance_to_actual_label[tuple(instance)]
            if actual_label == cluster_label:
                correct_assignments += 1
            total_assignments += 1
    accuracy = correct_assignments / total_assignments if total_assignments > 0 else 0
    return accuracy * 100

def compare_kmeans_accuracies(data, true_labels, k, metrics=["euclidean", "cosine", "jaccard"]):
    accuracies = {}
    for metric in metrics:
        print(f"\nRunning K-means with {metric} metric...")
        # clustering_result = kmeans.kmeans(data, k, metric=metric)
        clusters = results[metric]["clusters"]
        # Assign labels to clusters
        cluster_labels = label_clusters(clusters, true_labels, data)
        # Calculate accuracy
        accuracy = calculate_accuracy(clusters, cluster_labels, true_labels, data)
        accuracies[metric] = accuracy
        print(f"Accuracy for {metric}-K-means: {accuracy:.2f}%")
    # Determine the best metric
    best_metric = max(accuracies, key=accuracies.get)
    print(f"\nThe best metric is {best_metric} with an accuracy of {accuracies[best_metric]:.2f}%")
    return accuracies

metrics = ["euclidean", "cosine", "jaccard"]
accuracies = compare_kmeans_accuracies(data, label, k, metrics)


Running K-means with euclidean metric...
Accuracy for euclidean-K-means: 61.17%

Running K-means with cosine metric...
Accuracy for cosine-K-means: 61.24%

Running K-means with jaccard metric...
Accuracy for jaccard-K-means: 11.35%

The best metric is cosine with an accuracy of 61.24%


In [7]:
k = len(set(label))  # Number of clusters based on unique labels
metrics = ["euclidean", "cosine", "jaccard"]  # Metrics to compare

clustering_result_itr = {}
print("ITERATION")
for metric in metrics:
    res = kmeans.kmeans(data, k, metric=metric, stop_type="iteration")
    print(f"SSE for {metric}-K-means")
    
    clustering_result_itr[metric] = res
    print(f"iterations: {clustering_result_itr[metric]['iterations']}")
    print(f"time_taken: {clustering_result_itr[metric]['time_taken']}")

ITERATION
SSE for euclidean-K-means
iterations: 100
time_taken: 663.5364742279053
SSE for cosine-K-means
iterations: 100
time_taken: 551.1184651851654
SSE for jaccard-K-means
iterations: 100
time_taken: 172.26075553894043


In [8]:
k = len(set(label))  # Number of clusters based on unique labels
metrics = ["euclidean", "cosine", "jaccard"]  # Metrics to compare
clustering_result_centroid = {}
print("CENTROID")
for metric in metrics:
    res = kmeans.kmeans(data, k, metric=metric, stop_type="centroid")
    print(f"SSE for {metric}-K-means")
    
    clustering_result_centroid[metric] = res
    print(f"iterations: {clustering_result_centroid[metric]['iterations']}")
    print(f"time_taken: {clustering_result_centroid[metric]['time_taken']}")

CENTROID
SSE for euclidean-K-means
iterations: 81
time_taken: 534.8787603378296
SSE for cosine-K-means
iterations: 70
time_taken: 383.4009907245636
SSE for jaccard-K-means
iterations: 33
time_taken: 93.82068181037903


In [13]:
k = len(set(label))  # Number of clusters based on unique labels
metrics = [ "euclidean", "cosine", "jaccard" ]  # Metrics to compare
clustering_result_sse = {}
print("SSE")
for metric in metrics:
    res = kmeans.kmeans(data, k, metric=metric, stop_type="sse")
    print(f"SSE for {metric}-K-means")
    clustering_result_sse[metric] = res
    print(f"iterations: {clustering_result_sse[metric]['iterations']}")
    print(f"time_taken: {clustering_result_sse[metric]['time_taken']}")

SSE
SSE for euclidean-K-means
iterations: 121
time_taken: 813.3401138782501
SSE for cosine-K-means
iterations: 18
time_taken: 100.37496900558472
SSE for jaccard-K-means
iterations: 3
time_taken: 15.547428846359253


In [18]:
print("SSE Values for different Stopping Conditions")
print("---------------------------------------------")
print("Centroid")
for metric in metrics:
    
    print(metric.capitalize(), ":", clustering_result_centroid[metric]['withinss'])

print("---------------------------------------------")
print("Iterations (100)")
for metric in metrics:
    print(metric.capitalize(), ":",clustering_result_itr[metric]['withinss'])

print("---------------------------------------------")
print("SSE Increase")
for metric in metrics:
    print(metric.capitalize(), ":",clustering_result_sse[metric]['withinss'])

SSE Values for different Stopping Conditions
---------------------------------------------
Centroid
Euclidean : 25533557387.060757
Cosine : 2462.5295006930282
Jaccard : 9999.0
---------------------------------------------
Iterations (100)
Euclidean : 25578081537.80641
Cosine : 2473.0464869584493
Jaccard : 9999.0
---------------------------------------------
SSE Increase
Euclidean : 25318302482.14896
Cosine : 2472.954883111413
Jaccard : 9998.998091603053
